Demo of llama3 tool calling using OCI SDK

In [ ]:
import oci
def _init_oci_clients(profile_name):
    config = oci.config.from_file(profile_name=profile_name)
    generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config)
    generative_ai_client = oci.generative_ai.GenerativeAiClient(config)
    return generative_ai_inference_client,generative_ai_client


In [5]:
def set_user_message(prompt):
    command = oci.generative_ai_inference.models.SystemMessage()
    command.content = [
        oci.generative_ai_inference.models.TextContent(
            text = "Please, follow the next instructions: \n" \
            "You are a helpful assistant that answers questions from sales administrator \n " \
            "about the sales reps.Use the provided tools to search for data about the sales rep. \n " \
            "When searching, be persistent. If a tool result is not sufficient, suggest revised tool \n " \
            "parameters or another tool call."
        )
    ]
    # Step 1, call model with function definitions
    content = oci.generative_ai_inference.models.TextContent(
        text = prompt
    )

    user_message = oci.generative_ai_inference.models.UserMessage()
    user_message.content = [content]
    return user_message,command

In [6]:
#Actual function behind sales_search to return data.
def _function_sales_search():
    """"Actual login behind the search """
    return '"{\"sales_data\": \"1,000$\", \"firstName\": \"John\", \"month\": \"Jan\", \"year\": 2020}"'


In [7]:
#Tool function defenition
sales_search = oci.generative_ai_inference.models.FunctionDefinition(
    name="sales_search",
    description="Allows to search for sales rep data.",
    parameters = {
        "type": "object",
        "properties": {
            "firstName": {
                "type": "string",
            },
            "description": "FirstName of the sales rep."
        },
        "required": [ "firstName" ]

    })
    

In [17]:
def set_chat_params(command,user_message,mode,
                    tool_call_message:str=None,
                    tool_message:str=None,):
    chat_request = oci.generative_ai_inference.models.GenericChatRequest()
    chat_request.api_format = oci.generative_ai_inference.models.BaseChatRequest.API_FORMAT_GENERIC
    chat_request.temperature = 0
    chat_request.is_stream = False
    if mode == "set":
        chat_request.messages = [command, user_message]
        chat_request.tool_choice = oci.generative_ai_inference.models.ToolChoiceAuto()
        chat_request.tools = [ sales_search ]
    elif mode == "run":
        chat_request.messages = [command, user_message, tool_call_message, tool_message] 
    return chat_request

In [18]:
def set_oci_genai_chat_details(model_id,compartment_id,chat_request_details):
    chat_detail = oci.generative_ai_inference.models.ChatDetails()
    # Here to update to use desired model_id, more instruction in above TODO 2
    chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id=model_id)
    # chat_detail.serving_mode = oci.generative_ai_inference.models.DedicatedServingMode(endpoint_id="<your-endpoint-id>")
    chat_detail.compartment_id = compartment_id
    chat_detail.chat_request = chat_request_details
    return chat_detail

In [19]:
def run_chat_using_oci_geneai(oci_genai_chat_details,generative_ai_inference_client,message,verbose:bool=False):
    chat_response = generative_ai_inference_client.chat(oci_genai_chat_details)
    print(f"------------{message}---------------")
    if verbose == True:
        print(vars(chat_response))
    return chat_response

In [20]:
def parse_function_results_for_tool_executions(chat_response):
    #Step2 parse the tool function results back to model for final output
    tool_call_message = chat_response.data.chat_response.choices[0].message
    # TODO allow None content for tool call message
    tool_call_message.content = [oci.generative_ai_inference.models.TextContent(text="")]
    text = _function_sales_search()
    tool_content = oci.generative_ai_inference.models.TextContent(text = text )

    tool_message = oci.generative_ai_inference.models.ToolMessage(
        content = [ tool_content ],
        tool_call_id = tool_call_message.tool_calls[0].id
    )
    return tool_call_message, tool_message

In [ ]:
#Set Variables
COMPARTMENT_ID = "OCI Of the Compartment"
AUTH_TYPE = "API_KEY" # The authentication type to use, e.g., API_KEY (default), SECURITY_TOKEN, INSTANCE_PRINCIPAL, RESOURCE_PRINCIPAL.
CONFIG_PROFILE = "OCI PROFILE"
MODEL_ID="meta.llama-3.3-70b-instruct"
QUESTIONS=["What is the sales made by John for Jan 2020?","What is the total sales by Alex"]

In [13]:
#Disable proxy (Optional)
import os
os.environ['no_proxy'] = '*' 

In [21]:
generative_ai_inference_client,generative_ai_client = _init_oci_clients(profile_name=CONFIG_PROFILE)
count = 0 
for question in QUESTIONS:
    count += 1
    print(f"❓Ask {count}: -> {question}")
    user_message,command = set_user_message(prompt=question)
    chat_request_details = set_chat_params(command=command,user_message=user_message,mode="set")
    oci_genai_chat_details = set_oci_genai_chat_details(model_id=MODEL_ID,compartment_id=COMPARTMENT_ID,chat_request_details=chat_request_details)
    chat_response_from_step1 = run_chat_using_oci_geneai(
        oci_genai_chat_details=oci_genai_chat_details,
        generative_ai_inference_client=generative_ai_inference_client,
        message="❶ Step-1 set tool"
    )
    tool_call_message,tool_message=parse_function_results_for_tool_executions(chat_response=chat_response_from_step1)
    chat_request_details_for_tool_run = set_chat_params(command=command,user_message=user_message,mode="run",
                                                        tool_call_message=tool_call_message,tool_message=tool_message)
    oci_genai_chat_details.chat_request = chat_request_details_for_tool_run
    chat_response_from_step2 = run_chat_using_oci_geneai(
        oci_genai_chat_details=oci_genai_chat_details,
        generative_ai_inference_client=generative_ai_inference_client,
        message="❷ Step-2 run tool",
        verbose=True
    )

❓Ask 1: -> What is the sales made by John for Jan 2020?
------------❶ Step-1 set tool---------------
------------❷ Step-2 run tool---------------
{'status': 200, 'headers': {'content-type': 'application/json', 'opc-request-id': '5126EDF9701D466FB10112678FE12489/04CF875A20F2FAF4E7C9F8FD6A17D385/491DA8D9DE53BA57B08E3679BD27C4AF', 'content-encoding': 'gzip', 'content-length': '315'}, 'data': {
  "chat_response": {
    "api_format": "GENERIC",
    "choices": [
      {
        "finish_reason": "stop",
        "index": 0,
        "logprobs": {
          "text_offset": null,
          "token_logprobs": null,
          "tokens": null,
          "top_logprobs": null
        },
        "message": {
          "content": [
            {
              "text": "The sales made by John for Jan 2020 is $1,000.",
              "type": "TEXT"
            }
          ],
          "name": null,
          "role": "ASSISTANT",
          "tool_calls": []
        }
      }
    ],
    "time_created": "2025-04-2